# _Trip-Chaining_ _Method_ #

Aplica o _Trip-Chaining_ _Method_ de modo a gerar uma Matriz Origem-Destino a partir de uma tabela (Tabela I) com as entradas numa rede de transportes públicos.

Tabela I - Validations:

| Linha      | Paragem | Dia        | Hora     | NCliente |
| ---------- |-------  | ---------- | -------- | -------  |
| 714-0-ASC  | 18102   | 2018-01-07 | 15:00:39 | 1000     |
| 764-0-DESC | 13508   | 2018-01-14 | 09:45:15 | 2000     |
| 730-0-ASC  | 7006    | 2018-01-18 | 14:54:56 | 3000     |

Este método assume que cada entrada na rede é a origem de uma viagem realizada por uma pessoa e que a entrada seguinte que o mesmo indivíduo faz na rede é o destino da mesma viagem. Esta assumpção é depois confirmada e validada, verificando-se se a entrada "seguinte" num determinado local pode ser destino da entrada anterior. Tal validação segue a seguinte lógica: se se verificar que a "paragem da entrada seguinte" se encontra a menos de X minutos de uma paragem que pertence à linha associada à entrada na rede, o destino é considerado válido e a ocorrência do par origem-destino é contabilizada, caso contrário, essa ocorrência do par origem-destino é descartada da análise.

A incrementação deste processo de validação dos destinos só é viável se existirem outras duas tabelas auxiliares: tabela com todas as combinações possíveis entre linhas e paragens (Tabela II); e outra tabela com a extensão geográfica que é possível percorrer a partir de uma paragem em X minutos (o número de minutos fica ao critério do utilizador aquando da produção desta tabela).

Tabela II - Linhas:

| Linha      | Paragem | Ordem | Geometria    |
| ---------- |-------  | ----- | ------------ |
| 12E-0-CIRC | 801     | 1     | PARAGEM_GEOM |
| 12E-0-CIRC | 817     | 2     | PARAGEM_GEOM |
| 12E-0-CIRC | ...     | N     | PARAGEM_GEOM |
| 15E-0-ASC  | 917     | 1     | PARAGEM_GEOM |
| 15E-0-ASC  | 5515    | 2     | PARAGEM_GEOM |
| 15E-0-ASC  | ...     | N     | PARAGEM_GEOM |

Tabela III - Isócronas:

| Paragem | Geometria     |
| ------- | ------------- |
| 2214    | ISOCRONA_GEOM |
| 2215    | ISOCRONA_GEOM |
| 2216    | ISOCRONA_GEOM |
| 2217    | ISOCRONA_GEOM |
| 5515    | ISOCRONA_GEOM |
| 2322    | ISOCRONA_GEOM |

<hr>

### Parâmetros de entrada:###

###### Parâmetros obrigatórios: ######

* **CON_PARAM** -> Dicionário com o valor dos elementos necessários para estabelecer uma conexão com uma base de dados do PostgreSQL.

```Python
CON_PARAM = {
    "HOST" : "NOME_DO_SERVIDOR_ONDE_O_POSTGRESQL_ESTA_CONFIGURADO",
    "PORT" : "PORTO_DO_SERVIDOR_ONDE_O_POSTGRESQL_ESTA_CONFIGURADO",
    "USER" : "NOME_UTILIZADOR_POSTGRESQL",
    "PASSWORD" : "PASSWORD_UTILIZADOR_POSTGRESQL",
    "DATABASE" : "NOME_BASE_DE_DADOS"
}
```

* **TABLES_SCHEMA** -> Dicionário com os nomes das tabelas e respectivos campos com a informação que será usada ao longo do processo.

```Python
TABLES_SCHEMA = {
    # Esquema da Tabela com informação sobre as entradas na rede - equivalente à Tabela I
    "VALIDATIONS" : {
        "TNAME"  : "nome_tabela",
        "DAY"    : "nome_do_campo_com_indicacao_dia", # O formato deve ser YYYY-MM-DD
        "TIME"   : "nome_do_campo_com_indicacao_hora_min_sec", # O formato deve ser HH24:MI:SS
        "HOUR"   : "nome_do_campo_com_indicacao_hora",
        "MINUTE" : "nome_do_campo_com_indicacao_minuto",
        "SECOND" : "nome_do_campo_com_indicacao_segundo",
        "ROUTE"  : "nome_do_campo_com_identificacao_da_linha",
        "STOP"   : "nome_do_campo_com_identificacao_da_paragem",
        "CLIENT" : "nome_do_campo_com_identificacao_do_cliente"
    },
    # Esquema da Tabela com todas as combinações entre linhas e paragens - equivalente à Tabela II
    "STOPS" : {
        "TNAME" : "nome_tabela",
        "ROUTE" : "nome_do_campo_com_identificacao_da_linha",
        "STOP"  : "nome_do_campo_com_identificacao_da_paragem",
        "GEOM"  : "nome_do_campo_com_geometria_paragem
    },
    # Equivalente à Tabela III - Isócronas a partir das paragens
    "ISOCHRONES" : {
        "TNAME" : "nome_tabela",
        "STOP"  : "nome_do_campo_com_identificacao_da_paragem",
        "GEOM"  : "nome_do_campo_com_geometria_isocrona
    }
}
```

* **TABELA_SAIDA** -> Caminho para a tabela de saída. Esta tabela deve ter extensão ".xlsx".

###### Parâmetros opcionais: ######

* **FILTRO_DIA** -> Lista com os dias que o processo deve considerar;

```Python
# Exemplo
FILTRO_DIA = ['2018-01-15', '2018-01-16', '2018-01-17', '2018-01-18', '2018-01-19']
```

* **FILTRO_TEMPO** -> Período horário a considerar;

```Python
# Exemplo - Assim, só as entradas ocorridas entre as 9:00 e as 10:00 serão consideradas
FILTRO_DIA = ['09:00:00', '10:00:00']
```

* **MANTEM_TABELAS_INTERMEDIAS** -> Se for _True_, as tabelas geradas ao longo do processo não serão apagadas.

### Interpretação dos resultados ###

O processo gera uma tabela como a apresentada de seguida:

| origin_route | origin_stop | next_origin | destination_stop | trips_number |
| -----------  |-----------  | ----------- | ---------------- | -----------  |
| 701-0-ASC    | 2007        | 1512        | 1604             | 1            |
| 701-0-ASC    | 13621       | 1602        | 2718             | 1            |
| 736-0-ASC    | 3812        | 1102        | 1103             | 6            |
| 736-0-ASC    | 3812        | 1812        | 1807             | 4            |

Na qual:

* **origin_route** diz respeito à Linha da Paragem de Origem;
* **origin_stop** diz respeito à Paragem de entrada (origem);
* **next_origin** diz respeito à Paragem da entrada seguinte;
* **destination_stop** diz respeito à Paragem de destino (Paragem da mesma linha da Paragem de origem);
* **trips_number** diz respeito ao número de viagens realizadas entre determinada par Origem-Destino.

### Dependências ###

* **GIS Software **:
    * PostgreSQL;

* **Python Package**:
    * psycopg2;
    * Pandas;
    * xlsxwriter.

In [ ]:
from glass.transmob.matrixod import trip_chain_meth

CON_PARAM = {
    "HOST" : "localhost", "PORT" : "5432", "PASSWORD" : "admin",
    "USER" : "postgres", "DATABASE" : "valcarris_v2"
}

FILTRO_DIA = [
    #'2018-01-08', '2018-01-09', '2018-01-10', '2018-01-11', '2018-01-12', '2018-01-15',
    #'2018-01-16', '2018-01-17', '2018-01-18', '2018-01-19', '2018-01-22', '2018-01-23',
    #'2018-01-24', '2018-01-25', '2018-01-26', '2018-01-29', '2018-01-30', '2018-01-31'
    '2018-01-18'
]

FILTRO_TEMPO = ['09:00:00', '10:00:00']

TABLES_SCHEMA = {
    "VALIDATIONS" : {
        "TNAME"  : "idparesod",
        "DAY"    : "dia",
        "TIME"   : "tempo",
        "HOUR"   : "hora",
        "MINUTE" : "minuto",
        "SECOND" : "segundo",
        "ROUTE"  : "fid_percurso",
        "STOP"   : "paragem",
        "CLIENT" : "ncartao"
    },
    "ISOCHRONES" : {
        "TNAME"  : "isocronas",
        "STOP"   : "id",
        "GEOM"   : "geom"
    },
    "STOPS" : {
        "TNAME" : "percursos_geom_v2",
        "ROUTE" : "fid_percurso",
        "STOP"  : "paragem",
        "GEOM"  : "geom"
    }
}

MANTEM_TABELAS_INTERMEDIAS = None

TABELA_SAIDA = r'D:\TRENMO_JASP\matrixod_v18012018_9as10.xlsx'

trip_chain_meth(CON_PARAM, TABLES_SCHEMA, TABELA_SAIDA, FILTER_DAY=FILTER_DAY, FILTER_TIME=FILTER_TIME,
               saveTempTables=MANTEM_TABELAS_INTERMEDIAS)

In [ ]:
from glass.pgsql.mng.db       import create_db
from glass.pgsql.tbl          import copy_fromdb_todb
from glass.transmob.gtfs.circ import name_circulations
from glass.pgsql.anls.count   import sel_where_groupByIs
from glass.pgsql.tbl.w        import update_table
from glass.pgsql.tbl.w        import q_to_ntbl
from glass.pgsql.mng.fld      import drop_column
from glass.pgsql.tbl.w        import replace_null_with_other_col_value
from glass.pgsql.tbl._del     import drop_where_cols_are_same
from glass.toxls              import pgsql_to_xls

In [ ]:
conGTFS_DB = {
    "HOST" : "localhost", "PORT" : "5432", "PASSWORD" : "admin",
    "USER" : "postgres", "DATABASE" : "gtfs_carris"
}

CONEXAO_VALIDACOES_DB = {
    "HOST" : "localhost", "PORT" : "5432", "PASSWORD" : "admin",
    "USER" : "postgres", "DATABASE" : "valcarris"
}

NEW_DB = 'matrix_carris_1801'

GTFS_SCHEMA = {
    "STOP_TIMES" : {
        "TNAME"     : "stop_times",
        "TRIP"      : "trip_id",
        "STOP"      : "stop_id",
        "SEQUENCE"  : "stop_sequence",
        "DEPARTURE" : "departure_time"
    },
    "TRIPS" : {
        "TNAME"   : "trips",
        "TRIP"    : "trip_id",
        "SERVICE" : "service_id"
    },
    "CALENDAR" : {
        "TNAME"   : "calendar_dates",
        "SERVICE" : "service_id",
        "DATE"    : "date"
    }
}

VALIDACOES_SCHEMA = {
    "ENTRANCES" : {
        "TNAME"    : "validacoes_v9",
        "ROUTE"    : "route_id",
        "STOP"     : "paragem",
        "TIME"     : "time",
        "CAR"      : "nfrota",
        "DAY"      : "dia",
        "SEQUENCE" : "ordem",
        "CLIENT"   : "ncartao"
    },
    "STOPS" : {
        "TNAME"    : "percursos_geom_v2",
        "ROUTE"    : "fid_percurso",
        "SEQUENCE" : "ordem",
        "STOP"     : "paragem",
        "GEOM"     : "geom"
    },
    "ISOCHRONES" : {
        "TNAME"  : "isocronas",
        "STOP"   : "id",
        "GEOM"   : "geom"
    },
    "VAL_PROXIMITY" : {
        "TNAME"      : "dest_validation",
        "ROUTE_A"    : "fid_percurso",
        "STOP_B"     : "stop_b",
        "INTERSECTS" : "bintersect"
    },
    "NEAR_STOPS" : {
        "TNAME"  : "near_stops",
        "ROUTE"  : "fid_percurso",
        "STOP_A" : "stop_a",
        "STOP_B" : "stop_b",
        "DIST"   : "distance"
    }
}

# Two GTFS Days
GTFS_DAY = {20180307 : '2018-01-17', 20180308 : "2018-01-18"}
ENTRANCES_DAY = "2018-01-18"

OUTPUT = r'C:\gis\CARRIS\matrixod_1801.xlsx'

In [ ]:
from glass.transmob.bustops import get_isValidDestination_table
from glass.transmob.bustops import get_nearStopTable

destIsValidTbl = get_isValidDestination_table(
    CONEXAO_VALIDACOES_DB, VALIDACOES_SCHEMA["STOPS"]["TNAME"],
    VALIDACOES_SCHEMA["STOPS"]["STOP"],
    VALIDACOES_SCHEMA["STOPS"]["GEOM"],
    VALIDACOES_SCHEMA["STOPS"]["ROUTE"],
    VALIDACOES_SCHEMA["ISOCHRONES"]["TNAME"],
    VALIDACOES_SCHEMA["ISOCHRONES"]["STOP"],
    VALIDACOES_SCHEMA["ISOCHRONES"]["GEOM"],
    "dest_validation"
)

nearStopTable = get_nearStopTable(CONEXAO_VALIDACOES_DB,
    VALIDACOES_SCHEMA["STOPS"]["TNAME"],
    VALIDACOES_SCHEMA["STOPS"]["STOP"],
    VALIDACOES_SCHEMA["STOPS"]["GEOM"],
    VALIDACOES_SCHEMA["STOPS"]["ROUTE"],
    "near_stops"
)

In [ ]:
# We have two databases: GTFS AND other with database entrances
# Send all Relevant data to the same database
# Create a new one
conParam = {
    "HOST" : conGTFS_DB["HOST"], "PORT" : conGTFS_DB["PORT"],
    "PASSWORD" : conGTFS_DB["PASSWORD"], "USER" : conGTFS_DB["USER"]
}
create_db(conParam, NEW_DB, overwrite=True)
conParam["DATABASE"] = NEW_DB

# Copy GTFS Tables
copy_fromdb_todb(conGTFS_DB, conParam,
    [GTFS_SCHEMA[k]["TNAME"] for k in GTFS_SCHEMA]
)

copy_fromdb_todb(CONEXAO_VALIDACOES_DB, conParam,
    [VALIDACOES_SCHEMA[k]["TNAME"] for k in VALIDACOES_SCHEMA],
    qForTbl={VALIDACOES_SCHEMA["ENTRANCES"]["TNAME"] : (
        "SELECT * FROM {} WHERE {}='{}'".format(
            VALIDACOES_SCHEMA["ENTRANCES"]["TNAME"],
            VALIDACOES_SCHEMA["ENTRANCES"]["DAY"],
            ENTRANCES_DAY
        )
    )}
)

service_schema = {
    "TRIPS" : GTFS_SCHEMA["TRIPS"], "CALENDAR" : GTFS_SCHEMA["CALENDAR"],
    "FILTER_DAY" : GTFS_DAY.keys()
}
    
id_circulations = name_circulations(
    conParam, GTFS_SCHEMA["STOP_TIMES"], VALIDACOES_SCHEMA["STOPS"],
    "circ_id", serviceSchema=service_schema,
    tripIdColName="fid"
)

# In stop_times, add meta about the route (add route id)
q = (
    "SELECT {sT}.{tripId}, {sT}.{stopId}, {sT}.{stopSq}, "
    "{sT}.{depTime}, {idCirc}.{idRoute}, trip_service.{calenDate} AS gtfs_day "
    "FROM {sT} LEFT JOIN {idCirc} "
    "ON {sT}.{tripId} = {idCirc}.fid "
    "INNER JOIN ("
        "SELECT {tripsTbl}.{tripsTripId}, {calenTbl}.{calenDate} "
        "FROM {tripsTbl} INNER JOIN {calenTbl} ON "
        "{tripsTbl}.{tripsServId} = {calenTbl}.{calenServId}"
    ") AS trip_service "
    "ON {sT}.{tripId} = trip_service.{tripsTripId} "
    "WHERE {whrDays}"
).format(
    sT          = GTFS_SCHEMA["STOP_TIMES"]["TNAME"],
    tripId      = GTFS_SCHEMA["STOP_TIMES"]["TRIP"],
    stopId      = GTFS_SCHEMA["STOP_TIMES"]["STOP"],
    stopSq      = GTFS_SCHEMA["STOP_TIMES"]["SEQUENCE"],
    depTime     = GTFS_SCHEMA["STOP_TIMES"]["DEPARTURE"],
    idCirc      = id_circulations,
    idRoute     = VALIDACOES_SCHEMA["STOPS"]["ROUTE"],
    tripsTbl    = GTFS_SCHEMA["TRIPS"]["TNAME"],
    tripsTripId = GTFS_SCHEMA["TRIPS"]["TRIP"],
    tripsServId = GTFS_SCHEMA["TRIPS"]["SERVICE"],
    calenTbl    = GTFS_SCHEMA["CALENDAR"]["TNAME"],
    calenServId = GTFS_SCHEMA["CALENDAR"]["SERVICE"],
    calenDate   = GTFS_SCHEMA["CALENDAR"]["DATE"],
    whrDays     = " OR ".join(["trip_service.{} = {}".format(
        GTFS_SCHEMA["CALENDAR"]["DATE"], d
    ) for d in GTFS_DAY])
)

fidStopsTime = q_to_ntbl(conParam, "stop_times_fid", q)
GTFS_SCHEMA["STOP_TIMES"]["TNAME"] = fidStopsTime

# Sanitize DEPARTURE_TIME Field
for d in GTFS_DAY:
    update_table(conParam, GTFS_SCHEMA["STOP_TIMES"]["TNAME"], {
        GTFS_SCHEMA["STOP_TIMES"]["DEPARTURE"] : (
            "to_char(TO_TIMESTAMP('{day}' || ' ' || {depTime}, "
            "'YYYY-MM-DD HH24:MI:SS'), 'YYYY-MM-DD HH24:MI:SS')"
        ).format(
            day     = GTFS_DAY[d],
            depTime = GTFS_SCHEMA["STOP_TIMES"]["DEPARTURE"]
        )
    }, dic_ref_values={
        'gtfs_day' : d
    })

In [ ]:
# Sanitize time column in entrances table

update_table(conParam, VALIDACOES_SCHEMA["ENTRANCES"]["TNAME"], {
    VALIDACOES_SCHEMA["ENTRANCES"]["TIME"] : (
        "to_char(TO_TIMESTAMP({dayF} || ' ' || {tempo}, "
        "'YYYY-MM-DD HH24:MI:SS'), 'YYYY-MM-DD HH24:MI:SS')"
    ).format(
        dayF  = VALIDACOES_SCHEMA["ENTRANCES"]["DAY"],
        tempo = VALIDACOES_SCHEMA["ENTRANCES"]["TIME"]
    )
})

In [ ]:
# From bus entrances, erase cases when one card validate
# only one time in the same day
VALIDACOES_SCHEMA["ENTRANCES"]["TNAME"] = sel_where_groupByIs(
    conParam, VALIDACOES_SCHEMA["ENTRANCES"]["TNAME"],
    [VALIDACOES_SCHEMA["ENTRANCES"]["CLIENT"], VALIDACOES_SCHEMA["ENTRANCES"]["DAY"]],
    ">", "1", "p_{}".format(VALIDACOES_SCHEMA["ENTRANCES"]["TNAME"]),
    filterWhere="{}.{} = '{}'".format(
        VALIDACOES_SCHEMA["ENTRANCES"]["TNAME"],
        VALIDACOES_SCHEMA["ENTRANCES"]["DAY"], ENTRANCES_DAY
    )
)

In [ ]:
"""
Assign destinations to origins
"""

Q = (
    "SELECT *, "
    "first_value({routeId}) OVER (PARTITION BY {cardId}, {dayF} "
        "ORDER BY {dayF}, TO_TIMESTAMP({tempo}, 'YYYY-MM-DD HH24:MI:SS')) AS tmpv, "
    "first_value({stopId}) OVER (PARTITION BY {cardId}, {dayF} "
        "ORDER BY {dayF}, TO_TIMESTAMP({tempo}, 'YYYY-MM-DD HH24:MI:SS')) AS tmpst "
    "FROM ("
        "SELECT *, "
        "row_number() OVER(PARTITION BY {cardId}, {dayF} "
            "ORDER BY {cardId}, {dayF}, TO_TIMESTAMP({tempo}, "
            "'YYYY-MM-DD HH24:MI:SS')) AS valorder, "
        "lead({routeId}) OVER (PARTITION BY {cardId}, {dayF} "
            "ORDER BY {cardId}, {dayF}, TO_TIMESTAMP({tempo}, "
            "'YYYY-MM-DD HH24:MI:SS')) AS dest_route, "
        "lead({stopId}) OVER (PARTITION BY {cardId}, {dayF} "
            "ORDER BY {cardId}, {dayF}, TO_TIMESTAMP({tempo}, "
            "'YYYY-MM-DD HH24:MI:SS')) AS dest_stop "
        "FROM {t} ORDER BY {cardId}, {dayF}, "
        "TO_TIMESTAMP({tempo}, 'YYYY-MM-DD HH24:MI:SS')"
    ") AS sub_tbl"
).format(
    routeId = VALIDACOES_SCHEMA["ENTRANCES"]["ROUTE"],
    cardId  = VALIDACOES_SCHEMA["ENTRANCES"]["CLIENT"],
    dayF    = VALIDACOES_SCHEMA["ENTRANCES"]["DAY"],
    tempo   = VALIDACOES_SCHEMA["ENTRANCES"]["TIME"],
    stopId  = VALIDACOES_SCHEMA["ENTRANCES"]["STOP"],
    t       = VALIDACOES_SCHEMA["ENTRANCES"]["TNAME"]
)

matrix_od = q_to_ntbl(conParam, "p_matrix_od", Q)

# Update null values
# Last validation of the day does not have destiny
# This will be the first validation of the day
replace_null_with_other_col_value(conParam, matrix_od, 'dest_route', 'tmpv')
replace_null_with_other_col_value(conParam, matrix_od, 'dest_stop', 'tmpst')

drop_column(conParam, matrix_od, ["tmpv", "tmpst"])

# It is possible to find rows in which
# origins and destinations are the same
# Delete rows where this happens
drop_where_cols_are_same(conParam, matrix_od, VALIDACOES_SCHEMA["ENTRANCES"]["STOP"], "dest_stop")

In [ ]:
# Get a table with the potencial stop_time for one validation
# For a entrance with IDROUTE 26B|0|ASC will be created a row
# for each stop_time of the IDROUTE 26B|0|ASC

Q = (
    "SELECT * FROM ("
    "SELECT *, CASE "
        "WHEN tinterval_abs = MIN(tinterval_abs) OVER (PARTITION BY "
            "{valRou}, {card}, {stop}, {tempo}) "
        "THEN 1 ELSE 0 "
    "END AS iscirc FROM ("
        "SELECT * FROM ("
            "SELECT *, ("
                "(EXTRACT(day FROM dif_time) * 24) + "
                "EXTRACT(hour FROM dif_time) + "
                "(EXTRACT(minute FROM dif_time) / 60.0) + "
                "(EXTRACT(second FROM dif_time) / 3600.0)"
            ") AS tinterval, ABS("
                "(EXTRACT(day FROM dif_time) * 24) + "
                "EXTRACT(hour FROM dif_time) + "
                "(EXTRACT(minute FROM dif_time) / 60.0) + "
                "(EXTRACT(second FROM dif_time) / 3600.0)"
            ") AS tinterval_abs FROM ("
                "SELECT {valTbl}.{valRou}, {valTbl}.{stop}, {valTbl}.{seq}, {valTbl}.{day}, "
                "{valTbl}.{tempo}, {valTbl}.{card}, {valTbl}.valorder, {valTbl}.dest_route, "
                "{valTbl}.dest_stop, {stopTimeTbl}.{tripId}, "
                "{stopTimeTbl}.{depTime}, TO_TIMESTAMP({valTbl}.{tempo}, "
                "'YYYY-MM-DD HH24:MI:SS') - TO_TIMESTAMP({stopTimeTbl}.{depTime}, "
                "'YYYY-MM-DD HH24:MI:SS') AS dif_time "
                "FROM {valTbl} INNER JOIN {stopTimeTbl} ON "
                "{valTbl}.{valRou} = {stopTimeTbl}.{stopsRou} AND "
                "{valTbl}.{stop} = {stopTimeTbl}.{stopId} AND "
                "{valTbl}.{seq} = {stopTimeTbl}.{stopTSeq} "
                "WHERE (TO_TIMESTAMP({valTbl}.{tempo}, 'YYYY-MM-DD HH24:MI:SS') - "
                    "TO_TIMESTAMP({stopTimeTbl}.{depTime}, 'YYYY-MM-DD HH24:MI:SS')) < "
                    "(TO_TIMESTAMP('01:00:00', 'HH24:MI:SS') - TO_TIMESTAMP('00:15:00', 'HH24:MI:SS')) "
                    "AND (TO_TIMESTAMP({valTbl}.{tempo}, 'YYYY-MM-DD HH24:MI:SS') - "
                    "TO_TIMESTAMP({stopTimeTbl}.{depTime}, 'YYYY-MM-DD HH24:MI:SS')) > "
                    "(TO_TIMESTAMP('00:30:00', 'HH24:MI:SS') - TO_TIMESTAMP('01:00:00', 'HH24:MI:SS'))"
            ") AS val_stop_time"
        ") AS tmp_dif "
        "WHERE tinterval >= -0.1667 AND tinterval < 0.55"
    ") AS tmp_diff) AS foo WHERE iscirc = 1"
).format(
    valTbl      = matrix_od,
    valRou      = VALIDACOES_SCHEMA["ENTRANCES"]["ROUTE"],
    stop        = VALIDACOES_SCHEMA["ENTRANCES"]["STOP"],
    tempo       = VALIDACOES_SCHEMA["ENTRANCES"]["TIME"],
    stopTimeTbl = GTFS_SCHEMA["STOP_TIMES"]["TNAME"],
    tripId      = GTFS_SCHEMA["STOP_TIMES"]["TRIP"],
    stopId      = GTFS_SCHEMA["STOP_TIMES"]["STOP"],
    depTime     = GTFS_SCHEMA["STOP_TIMES"]["DEPARTURE"],
    stopsRou    = VALIDACOES_SCHEMA["STOPS"]["ROUTE"],
    seq         = VALIDACOES_SCHEMA["ENTRANCES"]["SEQUENCE"],
    card        = VALIDACOES_SCHEMA["ENTRANCES"]["CLIENT"],
    stopTSeq    = GTFS_SCHEMA["STOP_TIMES"]["SEQUENCE"],
    day         = VALIDACOES_SCHEMA["ENTRANCES"]["DAY"]
)

matrix_od = q_to_ntbl(conParam, "matrix_stoptime", Q)

In [ ]:
"""
Delete cases when one entrance has more than
one stop_time with a equal Minimum difference between
the validation time
"""

q = (
    "SELECT main_tbl.* "
    "FROM {valtbl} AS main_tbl INNER JOIN ("
        "SELECT {rouId}, {stop}, {tempo}, {card}, COUNT({stop}) AS conta "
        "FROM {valtbl} "
        "GROUP BY {rouId}, {stop}, {card}, {tempo}"
    ") AS foo "
    "ON main_tbl.{rouId} = foo.{rouId} AND main_tbl.{stop} = foo.{stop} "
    "AND main_tbl.{tempo} = foo.{tempo} AND main_tbl.{card} = foo.{card} "
    "WHERE (foo.conta > 1 AND main_tbl.tinterval < main_tbl.tinterval_abs) "
    "OR foo.conta = 1"
).format(
    rouId  = VALIDACOES_SCHEMA["ENTRANCES"]["ROUTE"],
    stop   = VALIDACOES_SCHEMA["ENTRANCES"]["STOP"],
    tempo  = VALIDACOES_SCHEMA["ENTRANCES"]["TIME"],
    card   = VALIDACOES_SCHEMA["ENTRANCES"]["CLIENT"],
    valtbl = matrix_od
)

matrix_od = q_to_ntbl(conParam, "matrix_stoptime_clean", q)

In [ ]:
# Check if destination is valid

matrix_od = q_to_ntbl(conParam, "p_matrix_od_valid", (
    "SELECT {mtx}.*, {valT}.bintersect "
    "FROM {mtx} INNER JOIN {valT} "
    "ON {mtx}.{routeF} = {valT}.{stopRouteF} AND "
    "{mtx}.dest_stop = {valT}.{stop_b} "
    "WHERE bintersect = 1"
).format(
    routeF     = VALIDACOES_SCHEMA["ENTRANCES"]["ROUTE"],
    stopRouteF = VALIDACOES_SCHEMA["VAL_PROXIMITY"]["ROUTE_A"],
    mtx        = matrix_od,
    valT       = VALIDACOES_SCHEMA["VAL_PROXIMITY"]["TNAME"],
    stop_b     = VALIDACOES_SCHEMA["VAL_PROXIMITY"]["STOP_B"]
))

In [ ]:
# Find real destinations

# Atencao: destino real nao pode ser igual a origem
# Se acontecer e porque a validacao seguinte nao deve ser
# o destino da primeira (paragens muito proximas)
matrix_od = q_to_ntbl(conParam, "p_matrix_od_realdest", (
    "SELECT mtx.{valRoute} AS origin_route, "
    "mtx.{valStop} AS origin_stop, "
    "mtx.dest_stop AS next_origin, "
    "neartbl.{stop_a} AS destination_stop, "
    "mtx.{tempo} AS origin_time, "
    "mtx.{depTime} AS theoric_origin_time, "
    "mtx.{sequence} AS stop_order, "
    "mtx.{tripId} AS trip_id, "
    "mtx.{card} AS nclient "
    "FROM {matrix} AS mtx "
    "INNER JOIN {nearStops} AS neartbl ON "
    "mtx.{valRoute} = neartbl.{stopsRoute} AND "
    "mtx.dest_stop = neartbl.{stop_b} "
    "WHERE mtx.{valStop} <> neartbl.{stop_a}"
).format(
    nearStops  = VALIDACOES_SCHEMA["NEAR_STOPS"]["TNAME"],
    valRoute   = VALIDACOES_SCHEMA["ENTRANCES"]["ROUTE"],
    valStop    = VALIDACOES_SCHEMA["ENTRANCES"]["STOP"],
    stop_a     = VALIDACOES_SCHEMA["NEAR_STOPS"]["STOP_A"],
    matrix     = matrix_od,
    stopsRoute = VALIDACOES_SCHEMA["NEAR_STOPS"]["ROUTE"],
    stop_b     = VALIDACOES_SCHEMA["NEAR_STOPS"]["STOP_B"],
    tempo      = VALIDACOES_SCHEMA["ENTRANCES"]["TIME"],
    sequence   = VALIDACOES_SCHEMA["ENTRANCES"]["SEQUENCE"],
    tripId     = GTFS_SCHEMA["STOP_TIMES"]["TRIP"],
    depTime    = GTFS_SCHEMA["STOP_TIMES"]["DEPARTURE"],
    card       = VALIDACOES_SCHEMA["ENTRANCES"]["CLIENT"]
))

In [ ]:
# Find arrival time

matrix_od = q_to_ntbl(conParam, "p_matrix_od_final", (
    "SELECT mtx.*, "
    "CASE "
        "WHEN TO_TIMESTAMP(stoptimes.{depTime}, 'YYYY-MM-DD HH24:MI:SS') > "
        "TO_TIMESTAMP(mtx.theoric_origin_time, 'YYYY-MM-DD HH24:MI:SS') "
        "THEN stoptimes.{depTime} "
        "ELSE CASE "
            "WHEN TO_TIMESTAMP(stoptimes.nextdeptime, 'YYYY-MM-DD HH24:MI:SS') "
            "> TO_TIMESTAMP(mtx.theoric_origin_time, 'YYYY-MM-DD HH24:MI:SS') "
            "THEN stoptimes.nextdeptime "
            "ELSE CASE "
                "WHEN TO_TIMESTAMP(stoptimes.nextofnext, "
                "'YYYY-MM-DD HH24:MI:SS') > TO_TIMESTAMP("
                "mtx.theoric_origin_time, 'YYYY-MM-DD HH24:MI:SS') "
                "THEN stoptimes.nextofnext ELSE 'notfound' "
            "END "
        "END "
    "END AS theoric_dest_arrival FROM ("
        "SELECT *, split_part(origin_time, ' ', 1) AS st_dia "
        "FROM {matrixOd}"
    ") AS mtx INNER JOIN ("
        "SELECT *, split_part({depTime}, ' ', 1) AS st_dia, "
        "lead({depTime}) OVER (PARTITION BY {routeF}, gtfs_day, "
            "{stopId}, {stopSeq} ORDER BY {stopId}, {stopSeq}, "
            "TO_TIMESTAMP({depTime}, 'YYYY-MM-DD HH24:MI:SS')) AS nextdeptime, "
        "lead({depTime}, 2) OVER (PARTITION BY {routeF}, gtfs_day, "
            "{stopId}, {stopSeq} ORDER BY {stopId}, {stopSeq}, "
            "TO_TIMESTAMP({depTime}, 'YYYY-MM-DD HH24:MI:SS')) AS nextofnext "
        "FROM {stopTimeTable}"
    ") AS stoptimes ON mtx.trip_id = stoptimes.{tripId} AND "
    "mtx.destination_stop = stoptimes.{stopId} AND "
    "mtx.st_dia = stoptimes.st_dia"
).format(
    depTime       = GTFS_SCHEMA["STOP_TIMES"]["DEPARTURE"],
    matrixOd      = matrix_od,
    routeF        = VALIDACOES_SCHEMA["STOPS"]["ROUTE"],
    stopId        = GTFS_SCHEMA["STOP_TIMES"]["STOP"],
    stopSeq       = GTFS_SCHEMA["STOP_TIMES"]["SEQUENCE"],
    stopTimeTable = GTFS_SCHEMA["STOP_TIMES"]["TNAME"],
    tripId        = GTFS_SCHEMA["STOP_TIMES"]["TRIP"]
))

In [ ]:
# If error in GTFS caused by the same trip ID related with different stops sequences:
# Group by and mantain the minor theoric_dest_arrival

matrix_od = q_to_ntbl(conParam, "p_matrix_od_final2", (
    "SELECT origin_route, origin_stop, next_origin, destination_stop, "
    "origin_time, theoric_origin_time, stop_order, trip_id, nclient, "
    "st_dia, MIN(theoric_dest_arrival) AS theoric_dest_arrival FROM {} "
    "GROUP BY origin_route, origin_stop, next_origin, destination_stop, "
    "origin_time, theoric_origin_time, stop_order, trip_id, nclient, "
    "st_dia"
).format(matrix_od))

In [ ]:
# Export result
pgsql_to_xls(matrix_od, OUTPUT, dic_con=conParam)